# Scrape for occupation 

format examples
- ANGUS STEWART, Crofter, Beinn-a-chorrain—examined.
- JOHN NICOLSON, Crofter and Fisherman, Cuillimore, Sconser—examined.
- DONALD MARTIN, Crofter, Kylerhea (65), assisted by MALCOLM MACPHERSON (about 60)—examined.
- ALEXANDER M'CASKILL, Cottar and Boatman, Soay (53)—examined.

-----
- format is in <b> tags
- Parsing for name, and town name 
- Split
- Save in csv

In [1]:
import pandas as pd
import requests
import re
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

In [2]:
index = pd.read_csv("../data/raw/napier_index.csv", encoding='windows-1252')
index = index.dropna()

In [3]:
def extract_first_header(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Find the first header (you can specify the header level, e.g., bold, etc.)
    first_header = soup.find(['b'])  # Adjust header levels as needed

    if first_header:
        return first_header.text.strip()
    else:
        return None  # Return None if no header is found

In [4]:
extract_first_header(index.iloc[65]["Url"])

"ALEXANDER M'INNES, Crofter, Kilmuir"

In [15]:
def make_record(row):
    header = extract_first_header(row["Url"])
    # if type(header) is tuple:
    #     return header
    # for col in index.columns[:-1]:
    #     header[col] = row[col]
    return header

In [16]:
tqdm.pandas(desc="Parse")
headers = index.iloc[0:10].progress_apply(make_record, axis = 1)

Parse:   0%|          | 0/10 [00:00<?, ?it/s]

In [1]:
headers

NameError: name 'headers' is not defined

In [ ]:
# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(headers.tolist())

In [10]:
result_df.head()

,0
0,"ANGUS STEWART, Crofter, Beinn-a-chorrain"
1,"Mr ALEXANDER MACDONALD, Factor for Lord Macdonald"
2,ANGUS STEWART
3,"SAMUEL NICOLSON, Crofter, Balmeanach"
4,"NEIL MACPHERSON, Crofter and Fisherman, Gedent..."


In [17]:
def parse_header(header):
    # Check if dfs is None or not a string
    if header is None or not isinstance(header, str):
        return "Unknown", "Unknown", "Unknown"

    # Split the filename into parts using '-' as the delimiter
    parts = header.split(',')
    parts = [p for part in parts for p in re.split(r"(\(\d+\))", part)]
    return parts
    
    # Check if the split parts contain at least 4 elements
    # if len(parts) >= 3:
    #     fullname = parts[0]
    #     occupation = parts[1]
    #     placename = parts[2]
    #     # age = parts[3]
    # else:
    #     fullname = "Unknown"
    #     occupation = "Unknown"
    #     placename = "Unknown"
    #     # age = "Unknown"

    # return fullname, occupation, placename

In [111]:
re.split(r"(\(\d+\))", "This is a test")

['This is a test']

In [18]:
records = [parse_header(h) for h in headers]

In [19]:
records[1:5]

[['Mr ALEXANDER MACDONALD', ' Factor for Lord Macdonald'],
 ['ANGUS STEWART'],
 ['SAMUEL NICOLSON', ' Crofter', ' Balmeanach'],
 ['NEIL MACPHERSON', ' Crofter and Fisherman', ' Gedentailler']]

In [20]:
data = pd.DataFrame.from_records(records)

In [116]:
data.to_excel("data.xlsx", sheet_name="sheet1", index=False)

In [21]:
data

,0,1,2,3
0,ANGUS STEWART,Crofter,Beinn-a-chorrain,None
1,Mr ALEXANDER MACDONALD,Factor for Lord Macdonald,None,None
2,ANGUS STEWART,None,None,None
3,SAMUEL NICOLSON,Crofter,Balmeanach,None
4,NEIL MACPHERSON,Crofter and Fisherman,Gedentailler,None
5,DONALD BUCHANAN,Crofter,Lower Olach,None
6,JOHN MATHESON,None,None,None
7,JOHN M'LEOD,Crofter and Fisherman,Camustionavaig,None
8,DONALD MACNEILL,Crofter,Glenmore,None
9,JOHN NICOLSON,Crofter and Fisherman,Cuillimore,Sconser


In [127]:
df= pd.read_excel("data_clean.xlsx")

In [136]:
# Check if the "Age" column exists
# Check if the DataFrame has at least four columns
if 'Age' in df.columns:
    # Remove round brackets from the "Age" column
    df['Age'] = df['Age'].astype(str).apply(lambda x: x.replace('(', '').replace(')', ''))

    # Check if the brackets are removed (optional debug)
    print("First few rows after cleaning 'Age' column:")
    print(df[['Age']])

    # Save the cleaned dataframe to a new Excel file
    output_file_path = 'output_file.xlsx'  # Replace with your desired output file path
    df.to_excel(output_file_path, index=False)
    print(f"Processed file saved to: {output_file_path}")
else:
    print("'Age' column not found in the sheet.")

First few rows after cleaning 'Age' column:
     Age
0    nan
1    nan
2    nan
3    nan
4    nan
..   ...
735   74
736  nan
737   66
738  nan
739   58

[740 rows x 1 columns]
Processed file saved to: output_file.xlsx


In [137]:
print("Columns in the sheet:", df.columns)

Columns in the sheet: Index(['Name', 'Occupation', 'Place', 'Age', 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='object')


# Cluster occupations

In [2]:
df= pd.read_excel("occupation_data.xlsx")

In [3]:
print("Columns in the sheet:", df.columns)

Columns in the sheet: Index(['Name', 'Occupation', 'Place', 'Age'], dtype='object')


In [4]:
df

,Name,Occupation,Place,Age
0,ANGUS STEWART,Crofter,Beinn-a-chorrain,NaN
1,Mr ALEXANDER MACDONALD,Factor for Lord Macdonald,NaN,NaN
2,ANGUS STEWART,NaN,NaN,NaN
3,SAMUEL NICOLSON,Crofter,Balmeanach,NaN
4,NEIL MACPHERSON,Crofter and Fisherman,Gedentailler,NaN
...,...,...,...,...
735,JOHN STUART BLACKIE,Emeritus-Professor of Greek in the University...,Edinburgh,74
736,JOHN MACKAY,Hereford,NaN,NaN
737,Sir WILLIAM COLLINS,Publisher,,66
738,ALEXANDER CARMICHAEL,30 Royal Circus,Edinburgh,NaN


- Landowners (Proprietor, Owner, Tacksman)

- Small landowners (Crofters, Feuar)

- Land workers (Cottars, Farmers, Tenant)

- Professionals (Lawyer, Teacher, Surgeon, Doctor, Postmaster, Accountant, Advocate, Agent, Clerk, Banker, Chamberlain, Merchant, Professor, Esq., Esquire, Factor, Hotel-Keeper, Inspector, Journalist, Commissioner, Licentiate, Procurator-Fiscal, Schoolmaster, Solicitor, Sub-Factor, Sheriff, Sheriff-Officer, Sheriff-Substitute, Constable, Estate Constable, Officer, National Service (?), Manager, Supervisor,)

- Clergy (Catechist, Minister, Missionary, Pastor, Clergyman, Priest,)

- Workers/Labourers (Mason, Tailor (Taylor), Shoemaker, Blacksmith, Boat-builder, Butcher, Carpenter, Weaver, Draper, Grocer, Shipwright, Boatman, Contractor, Fisherman, Carrier, Fish-Curer, Joiner, Labourer, Fisher, Curer, Pavoir, Pettester,
Quarryman, Seaman, Sheperd, Shopman, Trader)

In [5]:
#group1
landowners = ["Proprietor", "Owner", "Tacksman"]

#group2
small_landowners = ["Crofters", "Feuar"]

#group3
land_workers = ["Cottars", "Farmers", "Tenant"]

#group4
professionals = ["Lawyer", "Teacher", "Surgeon", "Doctor", "Postmaster", "Accountant", "Advocate", "Agent", "Clerk", 
    "Banker", "Chamberlain", "Merchant", "Professor", "Esq.", "Esquire", "Factor", "Hotel-Keeper", "Inspector", "Journalist", 
    "Commissioner", "Licentiate", "Procurator-Fiscal", "Schoolmaster", "Solicitor", "Sub-Factor", "Sheriff", "Sheriff-Officer", "Sheriff-Substitute", 
    "Constable", "Estate Constable", "Officer", "National Service", "Manager", "Supervisor",]

#group5
clergy = ["Catechist", "Minister", "Missionary", "Pastor", "Clergyman", "Priest"]

#group6
workers = ["Mason", "Tailor", "Taylor", "Shoemaker", "Blacksmith", "Boat-builder", "Butcher", "Carpenter", "Weaver", "Draper", "Grocer", "Shipwright", 
    "Boatman", "Contractor", "Fisherman", "Carrier", "Fish-Curer", "Joiner", "Labourer", "Fisher", "Curer", "Pavoir", "Pettester",
    "Quarryman", "Seaman", "Sheperd", "Shopman", "Trader"]

groups = []

6 arrarys with group classifications 
take the classification 

IF l in landowners
THEN group landowners 
ASSIGN number 1 
IF l in small_landowners
THEN group landowners in number 2
CREATE column for groups 1-6

In [3]:
if 'Occupation' in df.columns:
    if l in landowners:
        groups.append(1)  # Group 1 for landowners
    else if l in small_landowners:
        groups.append(2)  # Group 2 for small landowners
    else if l in land_workers:
        groups.append(3)     
    else if l in professionals:
        groups.append(4)  
    else if l in clergy:
        groups.append(5)
    else if l in workers:
        groups.append(6)
    else:
        groups.append(0)  # Group 0 for others (if needed)
        
    #df['Occupation'] = df['Occupation'].astype(str)
    

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/jovyan/SICSS/SICSS_project/In[3]#1:5\[90mIn[3]:1:5[0;0m]8;;\
if '[48;2;120;70;70mOccupation[0;0m' in df.columns:
[90m#   └────────┘ ── [0;0m[91mcharacter literal contains multiple characters[0;0m

In [6]:
# Function to assign groups
def assign_group(occupation):
    if occupation in landowners:
        return 'Landowner'
    elif occupation in small_landowners:
        return 'Small Landowner'
    elif occupation in land_workers:
        return 'Land Worker'
    elif occupation in professionals:
        return 'Professional'
    elif occupation in clergy:
        return 'Clergy'
    elif occupation in workers:
        return 'Worker'
    else:
        return 'Other'  # Default category if no match is found

# Apply the function to create a new column for groups
df['Group'] = df['Occupation'].apply(assign_group)

# Display the updated DataFrame
print(df)


                       Name  \
0             ANGUS STEWART   
1    Mr ALEXANDER MACDONALD   
2             ANGUS STEWART   
3           SAMUEL NICOLSON   
4           NEIL MACPHERSON   
..                      ...   
735     JOHN STUART BLACKIE   
736             JOHN MACKAY   
737     Sir WILLIAM COLLINS   
738    ALEXANDER CARMICHAEL   
739     ALEXANDER ROBERTSON   

                                            Occupation              Place  \
0                                              Crofter   Beinn-a-chorrain   
1                            Factor for Lord Macdonald                NaN   
2                                                  NaN                NaN   
3                                              Crofter         Balmeanach   
4                                Crofter and Fisherman       Gedentailler   
..                                                 ...                ...   
735   Emeritus-Professor of Greek in the University...          Edinburgh   
736        

In [7]:
df

,Name,Occupation,Place,Age,Group
0,ANGUS STEWART,Crofter,Beinn-a-chorrain,NaN,Other
1,Mr ALEXANDER MACDONALD,Factor for Lord Macdonald,NaN,NaN,Other
2,ANGUS STEWART,NaN,NaN,NaN,Other
3,SAMUEL NICOLSON,Crofter,Balmeanach,NaN,Other
4,NEIL MACPHERSON,Crofter and Fisherman,Gedentailler,NaN,Other
...,...,...,...,...,...
735,JOHN STUART BLACKIE,Emeritus-Professor of Greek in the University...,Edinburgh,74,Other
736,JOHN MACKAY,Hereford,NaN,NaN,Other
737,Sir WILLIAM COLLINS,Publisher,,66,Other
738,ALEXANDER CARMICHAEL,30 Royal Circus,Edinburgh,NaN,Other


In [13]:

# Define the groups in a dictionary
occupation_groups = {
    'Landowner': ["Proprietor", "Owner", "Tacksman"],
    'Small Landowner': ["Crofter", "Feuar"],
    'Land Worker': ["Cottar", "Farmer", "Tenant"],
    'Professional': ["Lawyer", "Teacher", "Surgeon", "Doctor", "Postmaster", "Accountant", 
                    "Advocate", "Agent", "Clerk", "Banker", "Chamberlain", "Merchant", 
                    "Professor", "Esq.", "Esquire", "Factor", "Hotel-Keeper", "Inspector", 
                    "Journalist", "Commissioner", "Licentiate", "Procurator-Fiscal", 
                    "Schoolmaster", "Solicitor", "Sub-Factor", "Sheriff", "Sheriff-Officer", 
                    "Sheriff-Substitute", "Constable", "Estate Constable", "Officer", 
                    "National Service", "Manager", "Supervisor"],
    'Clergy': ["Catechist", "Minister", "Missionary", "Pastor", "Clergyman", "Priest"],
    'Worker': ["Mason", "Tailor", "Taylor", "Shoemaker", "Blacksmith", "Boat-builder", 
               "Butcher", "Carpenter", "Weaver", "Draper", "Grocer", "Shipwright", 
               "Boatman", "Contractor", "Fisherman", "Carrier", "Fish-Curer", "Joiner", 
               "Labourer", "Fisher", "Curer", "Pavoir", "Pettester", "Quarryman", 
               "Seaman", "Shepherd", "Shopman", "Trader"]
}

# Function to assign groups
def assign_group(occupation):
    for group, occupations in occupation_groups.items():
        if occupation in occupations:
            return group
    return 'Other'  # Default category if no match is found

# Loop through the DataFrame and assign groups
df['Group'] = df['Occupation'].apply(assign_group)

# Display the updated DataFrame
print(df)

                       Name  \
0             ANGUS STEWART   
1    Mr ALEXANDER MACDONALD   
2             ANGUS STEWART   
3           SAMUEL NICOLSON   
4           NEIL MACPHERSON   
..                      ...   
735     JOHN STUART BLACKIE   
736             JOHN MACKAY   
737     Sir WILLIAM COLLINS   
738    ALEXANDER CARMICHAEL   
739     ALEXANDER ROBERTSON   

                                            Occupation              Place  \
0                                              Crofter   Beinn-a-chorrain   
1                            Factor for Lord Macdonald                NaN   
2                                                  NaN                NaN   
3                                              Crofter         Balmeanach   
4                                Crofter and Fisherman       Gedentailler   
..                                                 ...                ...   
735   Emeritus-Professor of Greek in the University...          Edinburgh   
736        

In [11]:
df

,Name,Occupation,Place,Age,Group
0,ANGUS STEWART,Crofter,Beinn-a-chorrain,NaN,Other
1,Mr ALEXANDER MACDONALD,Factor for Lord Macdonald,NaN,NaN,Other
2,ANGUS STEWART,NaN,NaN,NaN,Other
3,SAMUEL NICOLSON,Crofter,Balmeanach,NaN,Other
4,NEIL MACPHERSON,Crofter and Fisherman,Gedentailler,NaN,Other
...,...,...,...,...,...
735,JOHN STUART BLACKIE,Emeritus-Professor of Greek in the University...,Edinburgh,74,Other
736,JOHN MACKAY,Hereford,NaN,NaN,Other
737,Sir WILLIAM COLLINS,Publisher,,66,Other
738,ALEXANDER CARMICHAEL,30 Royal Circus,Edinburgh,NaN,Other


In [12]:
df.to_excel("sorted_occupation.xlsx", sheet_name="sheet1", index=False)